<a href="https://colab.research.google.com/github/poppvk935/Astrophysics-Deep-Learning/blob/main/Kaggle_Dissections/Kaggle_Dis_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nnAudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd #panel data
import matplotlib.pyplot as plt #make graphics
from matplotlib.gridspec import GridSpec #axes on graphic
import numpy as np #Lineal Algebra
import warnings #ignore warnings
from random import shuffle
import tensorflow as tf #model and dataset
from sklearn.model_selection import train_test_split
from scipy import signal
warnings.filterwarnings("ignore")

train_labels = pd.read_csv("/kaggle/input/g2net-gravitational-wave-detection/training_labels.csv")
sample_submission = pd.read_csv("/kaggle/input/g2net-gravitational-wave-detection/sample_submission.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/g2net-gravitational-wave-detection/training_labels.csv'

In [ ]:
def id2path(idx, is_train = True):
  path = "/kaggle/input/g2net-gravitational-wave-detection"
  if is_train:
    path += "/train/" + idx[0] + "/" + idx[1] + "/" + idx[2] + "/" + idx + ".npy"
  else:
    path += "/test/" + idx[0] + "/" + idx[1] + "/" + idx[2] + "/" + idx + ".npy"
  return path


In [ ]:
#increase dimension with constant Q-transform

from nnAudio.Spectrogram import CQT1992v2
import torch

def increase_dimension(idx, is_train, transform = CQT1992v2(sr = 2048, hop_length = 64, fmin = 20, fmax = 500)):
  wave = np.load(id2path(idx, is_train))
  wave = np.concatenate(wave, axis = 0)
  bHP, aHP = signal.butter(8, (20,500), btype = 'bandpass', fs=2048) #i think fs is the size of the image or possibly the stride
  window = signal.tukey(4096*3, 0.2)
  wave *= window
  wave = signal.filtfilt(bHP, aHP, wave)
  wave = wave/np.max(wave)
  wave = torch.from_numpy(wave).float()
  image = transform(wave)
  image = np.array(image)
  image = np.transpose(image, (1,2,0))

  return image

CQT kernels created, time used = 0.0574 seconds


In [ ]:
increase_dimension(train_labels["id"][0], is_train = True) #call increase dimension on training label row id col 0

NameError: name 'train_labels' is not defined

In [ ]:
#visualize data
targets = train_labels[train_labels["target"] == 0 ]["id"].head(2) #targets is first two rows of id, the 0th row of target
no_targets = train_labels[train_labels["target"] == 1]["id"].head(2)
fig = plt.figure(figsize=(15,10))
gs = GridSpec(4,4,figure = fig)

for i, (target, no_target) in enumerate(zip(targets, no_targets)):
  #subplot param GW found
  ax1 = fig.add_subplot(gs[i*2,0])
  ax1.plot(np.load(id2path(target))[1,:], color = 'blue')
  ax1.set_title("GW Found - Ligo Hanford")
  ax1.set_axis_off()


  ax2 = fig.add_subplot(gs[i*2, 1])
  ax2.plot(np.load(id2path(target))[2,:], color='blue')
  ax2.set_title("GW Found - Ligo Livingstone")
  ax2.set_axis_off()

  ax3 = fig.add_subplot(gs[i*2, 2])
  ax3.plot(np.load(id2path(target))[0,:], color = 'blue')
  ax3.set_title("GW Found - Virgo")
  ax3.set_axis_off()

  ax4 = fig.add_subplot(gs[i*2, 3])
  ax4.imshow(increase_dimension(target, is_train=True))
  ax4.set_title("GW Found = Spectrogram")
  ax4.set_axis_off()

  #subplot param GW not found
  ax5 = fig.add_subplot(gs[i*2 +1, 0])
  ax5.plot(np.load(id2path(no_target))[1,:], color='red')
  ax5.set_title("GW Not Found - Ligo Hanford")
  ax5.set_axis_off()

  ax6 = fig.add_subplot(gs[i*2+1, 1])
  ax6.plot(np.load(id2path(no_target))[2,:], color='red')
  ax6.set_title("GW Not Found - Livingstone")
  ax6.set_axis_off()

  ax7 = fig.add_subplot(gs[i*2+1, 2])
  ax7.plot(np.load(id2path(no_target))[0,:], color='red')
  ax7.set_title("GW Not Found - Virgo")
  ax7.set_axis_off()

  ax8 = fig.add_subplot(gs[i*2+1, 3])
  ax8.imshow(increase_dimension(no_target, is_train=True))
  ax8.set_title("GW Not Found - Spectrogram")
  ax8.set_axis_off()

  plt.tight_layout()
  plt.show()

NameError: name 'train_labels' is not defined

In [ ]:
#Batch Data

import math
class Dataset(tf.keras.utils.Sequence):
  def __init__(self, data, y=None, batch_size = 256, shuffle=True): #data = inputs, no outputs
    self.data = data
    self.batch_size = batch_sizes
    self.shuffle = shuffle

    if y is not None:
      self.is_train = True
    else:
      self.is_train = False


    self.y = y

  def __len__(self):
    return math.ceil(len(self.data)/self.batch_size)

  def __getitem__(self, ids):
    batch_data = self.data[ids*self.batch_size : (ids + 1) * self.batch_size]

    if self.y is not None:
      batch_y = self.y[ids*self.batch_size : (ids+1)*self.batch_size]

    batch_x = np.array([increase_dimension(x, self.is_train), for x in batch_data])
    batch_x = np.stack(batch_x)

    if self.is_train:
      return batch_x, batch_y
    else:
      return batch_x
  def on_epoch_end(self):
    if self.shuffle and self.is_train:
      ids_y = list(zip(self.data, self.y))
      shuffle(ids_y)
      self.data, self.y = list(zip(*ids_y))

SyntaxError: did you forget parentheses around the comprehension target? (ipython-input-2350586768.py, line 27)

In [ ]:
train_idx = train_labels['id'].values
y = train_labels['target'].values
test_idx = sample_submission['id'].values

x_train, x_valid, y_train, y_valid = train_test_split(train_idx, y, test_size=0.05, random_state=42, stratify=y)

train_dataset = Dataset(x_train, y_train)
valid_dataset = Dataset(x_valid, y_valid)
test_dataset = Dataset(test_idx)

NameError: name 'train_labels' is not defined

In [ ]:
pip install efficientnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00


In [ ]:
def create_model():
  Conv1 = tf.keras.layers.Conv2d(filters = 32, kernel_size = (3,3), input_shape = [56, 193, 1], activation = "elu")
  Maxpooling1 = tf.keras.layers.MaxPool2d()
  Conv2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = "elu")
  Maxpooling2 = tf.keras.layers.MaxPool2D()
  Conv3 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = "elu")
  Maxpooling3 = tf.keras.layers.MaxPool2D()
  flatten = tf.keras.layers.Flatten()
  Dense1 = tf.keras.layers.Dense(64, activation = "relu")
  dense2 = tf.keras.layers.Dense(1, activation = "sigmoid")

  model = tf.keras.Sequential([Conv1, Maxpooling1, Conv2, Maxpooling2, Conv3, Maxpooling3, flatten, Dense1, dense2])
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = [tf.keras.metrics.AUC()])

  return model

model = create_model()

from tensorflow.keras.utils import plot_model

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = 'logs')

plot_model(model, to_file='model.png', show_shapes=true)

NameError: name 'tf' is not defined

In [ ]:
model.fit(train_dataset, epochs=2, validation_data = validation_dataset)

NameError: name 'model' is not defined

In [ ]:
preds = model.predict(test_dataset)
preds = preds.reshape(-1)